# 2018 General Election Midterm Candidate Scraper

In [2]:
from urllib.request import urlopen
import re # Regex Library
import pandas as pd

In [8]:
def get_html_from_url(url): 
    return urlopen(url).read().decode("utf-8")

In [9]:
states = []
with open('states.txt', 'r') as f:
    for state in f:
        states.append(state[:-1].replace(' ', '_'))

In [10]:
states

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New_Hampshire',
 'New_Jersey',
 'New_Mexico',
 'New_York',
 'North_Carolina',
 'North_Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode_Island',
 'South_Carolina',
 'South_Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West_Virginia',
 'Wisconsin',
 'Wyoming']

## House of Representatives

### Get Ballotpedia HTML for each state

I'm encountering a problem where some lists of general election candidates are formatted differently than others. I have identified 2 patterns: a more modern html approach where "General election candidates" is between b tags and candidate names are enclosed in a tags, and an older approach where "General election candidates" is enclosed in span tags and the candidate names are in ul tags.

In [11]:
def clean_names(possible_names):
    names = []
    for string in possible_names:
        if "election" not in string and "candidate" not in string:
                    names.append(string)
    return names

In [12]:
def get_candidates_for_state(state, chamber):
    chamber_urls = {'house': 'https://ballotpedia.org/United_States_House_of_Representatives_elections_in_{},_2018', 'senate': 'https://ballotpedia.org/United_States_Senate_election_in_{},_2018'}
    html = get_html_from_url(chamber_urls[chamber].format(state))
    districts = re.findall(r"District \d{1,2}.{1,300}General election.+?Primary", html, re.DOTALL)
#     print(districts)
    data = pd.DataFrame()
    if districts:
        for district in districts:
            district_string = re.findall(r"District \d{1,2}", district)[0]
            candidate_names = clean_names(re.findall(r"<a href=.+>(.+)</a>", district))
            data = data.append(pd.DataFrame({'district': [district_string]*len(candidate_names), 'name': candidate_names, 'state': state}), ignore_index=True)
    else:
        general_candidates = re.findall(r"General election candidates.+?Primary", html, re.DOTALL)
        candidate_names = clean_names(re.findall(r"<a href=.+>(.+)</a>", general_candidates[0]))
        data = data.append(pd.DataFrame({'district': [None]*len(candidate_names), 'name': candidate_names, 'state': state}), ignore_index=True)
    return data

In [279]:
get_candidates_for_state('California', 'senate')

,district,name,state
0,None,Early voting,California
1,None,Absentee voting deadline,California
2,None,Yes,California
3,None,Same-day registration,California
4,None,Voter ID,California
5,None,Poll times,California
6,None,Recalls,California


## Get House candidates for each state

In [272]:
df = pd.DataFrame()
for state in states:
    df = df.append(get_candidates_for_state(state, 'house'), ignore_index=True)
    print(get_candidates_for_state(state))

      district                name    state
0   District 1  Robert Kennedy Jr.  Alabama
1   District 1       Bradley Byrne  Alabama
2   District 2       Tabitha Isner  Alabama
3   District 2         Martha Roby  Alabama
4   District 3       Mallory Hagan  Alabama
5   District 3         Mike Rogers  Alabama
6   District 4           Lee Auman  Alabama
7   District 4     Robert Aderholt  Alabama
8   District 5      Peter Joffrion  Alabama
9   District 5           Mo Brooks  Alabama
10  District 6        Danner Kline  Alabama
11  District 6         Gary Palmer  Alabama
12  District 7        Terri Sewell  Alabama
  district          name   state
0     None     Don Young  Alaska
1     None  Alyse Galvin  Alaska
2     None    Greg Fitch  Alaska
3     None      Sid Hill  Alaska
      district                     name    state
0   District 1       Tom O&#39;Halleran  Arizona
1   District 1             Wendy Rogers  Arizona
2   District 1              David Shock  Arizona
3   District 2         

Empty DataFrame
Columns: [district, name, state]
Index: []
  district                  name     state
0     None  Lisa Blunt Rochester  Delaware
1     None          Scott Walker  Delaware
2     None           Andrew Webb  Delaware
3     None          Marvin Davis  Delaware
4     None         Paul Johnston  Delaware
       district                      name    state
0    District 2                 Neal Dunn  Florida
1    District 2              Bob Rackleff  Florida
2    District 3                  Ted Yoho  Florida
3    District 3       Yvonne Hayes Hinson  Florida
4    District 4           John Rutherford  Florida
5    District 4               Ges Selmont  Florida
6    District 4          Joceline Berrios  Florida
7    District 4              Jason Bulger  Florida
8    District 5             Alfred Lawson  Florida
9    District 5           Virginia Fuller  Florida
10   District 6           Nancy Soderberg  Florida
11   District 6             Michael Waltz  Florida
12   District 7     

      district                name    state
0   District 1     Peter Visclosky  Indiana
1   District 1          Mark Leyva  Indiana
2   District 1   Jonathan Kleinman  Indiana
3   District 2     Jackie Walorski  Indiana
4   District 2            Mel Hall  Indiana
5   District 2        Richard Wolf  Indiana
6   District 3           Jim Banks  Indiana
7   District 3     Courtney Tritch  Indiana
8   District 4           Tobi Beck  Indiana
9   District 4           Jim Baird  Indiana
10  District 5        Susan Brooks  Indiana
11  District 5        Dee Thornton  Indiana
12  District 6   Jeannine Lee Lake  Indiana
13  District 6          Greg Pence  Indiana
14  District 6         &#91;3&#93;  Indiana
15  District 6       Heather Meloy  Indiana
16  District 6         John Miller  Indiana
17  District 7        Andre Carson  Indiana
18  District 7        Wayne Harmon  Indiana
19  District 8       Larry Bucshon  Indiana
20  District 8      William Tanoos  Indiana
21  District 9  Trey Hollingswor

     district                 name  state
0  District 1      Chellie Pingree  Maine
1  District 1        Mark Holbrook  Maine
2  District 1       Martin Grohman  Maine
3  District 2       Bruce Poliquin  Maine
4  District 2         Jared Golden  Maine
5  District 2         Tiffany Bond  Maine
6  District 2            Will Hoar  Maine
7  District 2  Dennis O&#39;Connor  Maine
8  District 2  Danielle VanHelsing  Maine
9  District 2      Henry John Bear  Maine
      district                      name     state
0   District 1               Andy Harris  Maryland
1   District 1              Jesse Colvin  Maryland
2   District 1             Jenica Martin  Maryland
3   District 2       Dutch Ruppersberger  Maryland
4   District 2                Liz Matory  Maryland
5   District 2                Guy Mimoun  Maryland
6   District 2            Michael Carney  Maryland
7   District 3             John Sarbanes  Maryland
8   District 3           Charles Anthony  Maryland
9   District 3              

     district                  name       state
0  District 1         Debra Haaland  New_Mexico
1  District 1   Janice Arnold-Jones  New_Mexico
2  District 1       Lloyd Princeton  New_Mexico
3  District 2  Xochitl Torres Small  New_Mexico
4  District 2        Yvette Herrell  New_Mexico
5  District 3         Ben Ray Lujan  New_Mexico
6  District 3   Jerald Steve McFall  New_Mexico
7  District 3   Christopher Manning  New_Mexico
        district                                               name     state
0     District 1                                         Democratic  New_York
1     District 1                                         Republican  New_York
2     District 1                                 Conservative Party  New_York
3     District 1                                        Green Party  New_York
4     District 1                                 Independence Party  New_York
5     District 1                                       Reform Party  New_York
6     District 1      

  district             name         state
0     None    Mac Schneider  North_Dakota
1     None  Kelly Armstrong  North_Dakota
2     None   Charles Tuttle  North_Dakota
       district                          name state
0    District 1                  Steve Chabot  Ohio
1    District 1                 Aftab Pureval  Ohio
2    District 1                   Dirk Kubala  Ohio
3    District 1              Mike Goldschmidt  Ohio
4    District 1                 Kiumars Kiani  Ohio
5    District 2                 Brad Wenstrup  Ohio
6    District 2                 Jill Schiller  Ohio
7    District 2              James Condit Jr.  Ohio
8    District 2                   Steve Myers  Ohio
9    District 2                   David Baker  Ohio
10   District 3                  Joyce Beatty  Ohio
11   District 3                   Jim Burgess  Ohio
12   District 3                  Millie Milam  Ohio
13   District 4                    Jim Jordan  Ohio
14   District 4                 Janet Garrett  Ohio


      district              name state
0   District 1        Rob Bishop  Utah
1   District 1      Lee Castillo  Utah
2   District 1        Adam Davis  Utah
3   District 1      Eric Eliason  Utah
4   District 2     Chris Stewart  Utah
5   District 2  Shireen Ghorbani  Utah
6   District 2   Jeffrey Whipple  Utah
7   District 2       Jan Garbett  Utah
8   District 3       John Curtis  Utah
9   District 3      James Singer  Utah
10  District 3       &#91;4&#93;  Utah
11  District 3   Gregory Duerden  Utah
12  District 3       &#91;4&#93;  Utah
13  District 4          Mia Love  Utah
14  District 4       Ben McAdams  Utah
  district             name    state
0     None      Peter Welch  Vermont
1     None       Anya Tynio  Vermont
2     None     Laura Potter  Vermont
3     None     Cris Ericson  Vermont
4     None  H. Brooke Paige  Vermont
5     None      &#91;1&#93;  Vermont
       district                   name     state
0    District 1            Rob Wittman  Virginia
1    District 1    

In [273]:
df.to_csv("candidate_names.csv")

## Get party affiliation and incumbency data for each candidate

We manually fixed the csv to get a complete list of candidates for Congress from each state, using Ballotpedia as our source. We will now add incumbency and party affiliation data to the csv we already have.

In [3]:
df = pd.read_csv('candidate_names_final.csv')

In [4]:
df

,district,name,state,chamber,party
0,District 1,Robert Kennedy Jr.,Alabama,house,NaN
1,District 1,Bradley Byrne,Alabama,house,NaN
2,District 2,Tabitha Isner,Alabama,house,NaN
3,District 2,Martha Roby,Alabama,house,NaN
4,District 3,Mallory Hagan,Alabama,house,NaN
5,District 3,Mike Rogers,Alabama,house,NaN
6,District 4,Lee Auman,Alabama,house,NaN
7,District 4,Robert Aderholt,Alabama,house,NaN
8,District 5,Peter Joffrion,Alabama,house,NaN
9,District 5,Mo Brooks,Alabama,house,NaN


Get House candidate info

In [5]:
house_df = df.loc[df.chamber == 'house']

In [6]:
house_df

,district,name,state,chamber,party
0,District 1,Robert Kennedy Jr.,Alabama,house,NaN
1,District 1,Bradley Byrne,Alabama,house,NaN
2,District 2,Tabitha Isner,Alabama,house,NaN
3,District 2,Martha Roby,Alabama,house,NaN
4,District 3,Mallory Hagan,Alabama,house,NaN
5,District 3,Mike Rogers,Alabama,house,NaN
6,District 4,Lee Auman,Alabama,house,NaN
7,District 4,Robert Aderholt,Alabama,house,NaN
8,District 5,Peter Joffrion,Alabama,house,NaN
9,District 5,Mo Brooks,Alabama,house,NaN


In [ ]:
for state in states:

In [13]:
chamber_urls = {'house': 'https://ballotpedia.org/United_States_House_of_Representatives_elections_in_{},_2018', 'senate': 'https://ballotpedia.org/United_States_Senate_election_in_{},_2018'}

In [15]:
html = get_html_from_url(chamber_urls['house'].format('California'))

In [16]:
html

'<!-- here -->\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>United States House of Representatives elections in California, 2018 - Ballotpedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"United_States_House_of_Representatives_elections_in_California,_2018","wgTitle":"United States House of Representatives elections in California, 2018","wgCurRevisionId":7529838,"wgRevisionId":7529838,"wgArticleId":980309,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with reference errors","No Republican candidates","No Democratic candidates","U.S. House elections, 2018","California elections, 2018"]

In [32]:
re.findall(r"Democratic Party.*?Diane Harkey", html)

['Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Douglas_Applegate" title="Douglas Applegate">Douglas Applegate</a><sup id="cite_ref-194" class="reference"><a href="#cite_note-194">&#91;194&#93;</a></sup><br /><a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Sara_Jacobs" title="Sara Jacobs">Sara Jacobs</a><sup id="cite_ref-195" class="reference"><a href="#cite_note-195">&#91;195&#93;</a></sup><br /><a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Paul_Kerr" title="Paul Kerr">Paul Kerr</a><sup id="cite_r

In [70]:
re.findall('<img alt="(.*?)".*?Diane Harkey', html)

['Republican Party', 'Democratic Party']

In [71]:
long = re.findall(r'<a.+?>.*?Diane Harkey', html)
long

['<a href="/Republican_Party" title="Republican Party"><img alt="Republican Party" src="https://cdn.ballotpedia.org/images/f/fd/Ends.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Diane_Harkey" title="Diane Harkey',
 '<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Douglas_Applegate" title="Douglas Applegate">Douglas Applegate</a><sup id="cite_ref-194" class="reference"><a href="#cite_note-194">&#91;194&#93;</a></sup><br /><a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Sara_Jacobs" title="Sara Jacobs">Sara Jacobs</a><sup id="cite_ref-195" class="reference"><a href="#cite_note-195">&#91;195&#93;</a></s

In [73]:
min(long, key=len)

'<a href="/Republican_Party" title="Republican Party"><img alt="Republican Party" src="https://cdn.ballotpedia.org/images/f/fd/Ends.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Diane_Harkey" title="Diane Harkey'

In [72]:
re.findall('<img alt="(.*?)".*?Diane Harkey', long[0])

['Republican Party']

In [33]:
re.findall(r"Republican Party.*?Diane Harkey", html)

['Republican Party"><img alt="Republican Party" src="https://cdn.ballotpedia.org/images/f/fd/Ends.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Diane_Harkey" title="Diane Harkey',
 'Republican Party"><img alt="Republican Party" src="https://cdn.ballotpedia.org/images/f/fd/Ends.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Rocky_Ch%C3%A1vez" title="Rocky Chávez">Rocky Chávez</a>, state assemblyman<sup id="cite_ref-repcandidates_198-0" class="reference"><a href="#cite_note-repcandidates-198">&#91;198&#93;</a></sup><br /><a href="/Republican_Party" title="Republican Party"><img alt="Republican Party" src="https://cdn.ballotpedia.org/images/f/fd/Ends.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Kristin_Gaspar" title="Kristin Gaspar">Kristin Gaspar</a>, San Diego County supervisor<sup id="cite_ref-199" class="reference"><a href="#cite_note-199">&#91;199&#93;</a></sup><br 

In [ ]:
        html_near_name = re.findall(r'<a.+?>.*?{}'.format(candidate_name), html)


In [117]:
def get_party(candidate_name, html):
    try:
        html_near_name = re.findall(r'<a.+?>.*?<img alt=".*?{}'.format(candidate_name), html)
        print(html_near_name)
        relevant_html = min(html_near_name, key=len)
        print(relevant_html)
        party = re.findall('<img alt="(.*?)".*?{}'.format(candidate_name), relevant_html)
        print(party)
        return party[0]
    except:
        print(candidate_name)
        return 'cannot determine'
    
#     parties = ['Democrat', 'Republican', 'Independent', 'Libertarian', 'Green']
#     dem_html = re.findall(r"Democratic Party.*?{}".format(candidate_name, html), html)
#     rep_html = re.findall(r"Republican Party.*?{}".format(candidate_name, html), html)
#     if dem_html:
#         if rep_html:
#             if len(dem_html[0]) < len(rep_html[0]):
#                 return 'D'
#             else:
#                 return 'R'
#         else:
#             return 'D'
#     else:
#         return 'R'

In [116]:
get_party('Audrey Denney', html)

['<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Audrey_Denney" title="Audrey Denney', '<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Audrey_Denney" title="Audrey Denney']
<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Audrey_Denney" title="Audrey Denney
['Democratic Party']


'Democratic Party'

In [98]:
ca_df = house_df.loc[house_df.state == 'California'].copy()

In [118]:
for name in ca_df.name:
    ca_df.loc[ca_df.name == name, 'party'] = get_party(name, html)

['<a href="/Republican_Party" title="Republican Party"><img alt="Republican Party" src="https://cdn.ballotpedia.org/images/f/fd/Ends.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Doug_LaMalfa" title="Doug LaMalfa', '<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Audrey_Denney" title="Audrey Denney">Audrey Denney</a><sup id="cite_ref-1" class="reference"><a href="#cite_note-1">&#91;1&#93;</a></sup> <a href="/Approved" title="Approved"><img alt="Approved" src="https://cdn.ballotpedia.org/images/7/77/Yes.png" width="16" height="16" data-file-width="16" data-file-height="16" /></a><span style="display:none;">a</span><br /><a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" hei

<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Jessica_Morse" title="Jessica Morse
['Democratic Party']
['<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Mike_Thompson_(California)" title="Mike Thompson', '<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Mike_Thompson_(California)" title="Mike Thompson', '<a href="/Template:California_congress" title="Template:California congress"><span title="View this template" style=";background:#bbccee;;border:none;;">v</span></a>

<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Jrmar_Jefferson" title="Jrmar Jefferson
['Democratic Party']
['<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Ami_Bera" title="Ami Bera', '<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Ami_Bera" title="Ami Bera', '<a href="/Template:California_congress" title="Template:California congress"><span title="View this template" style=";background:#bbccee;;border:none;;">v</span></a>&#160;<span style="font-size:80%;">•</span

<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Eric_Swalwell" title="Eric Swalwell
['Democratic Party']
['<a href="/Republican_Party" title="Republican Party"><img alt="Republican Party" src="https://cdn.ballotpedia.org/images/f/fd/Ends.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Rudy_Peters" title="Rudy Peters', '<a href="/Democratic_Party" title="Democratic Party"><img alt="Democratic Party" src="https://cdn.ballotpedia.org/images/d/d4/Electiondot.png" width="18" height="18" data-file-width="16" data-file-height="16"></a> <a href="/Eric_Swalwell" title="Eric Swalwell">Eric Swalwell</a> - Incumbent<sup id="cite_ref-51" class="reference"><a href="#cite_note-51">&#91;51&#93;</a></sup> <a href="/Approved" title="Approved"><img alt="Approved" src="https://cdn.ballotpedia.or

In [119]:
ca_df

,district,name,state,chamber,party
61,District 1,Doug LaMalfa,California,house,Republican Party
62,District 1,Audrey Denney,California,house,Democratic Party
63,District 2,Jared Huffman,California,house,Democratic Party
64,District 2,Dale Mensing,California,house,Republican Party
65,District 3,John Garamendi,California,house,Democratic Party
66,District 3,Charlie Schaupp,California,house,Republican Party
67,District 4,Tom McClintock,California,house,Republican Party
68,District 4,Jessica Morse,California,house,Democratic Party
69,District 5,Mike Thompson,California,house,Democratic Party
70,District 5,Anthony Mills,California,house,Independent


In [120]:
ca_df.to_csv('ca_candidates.csv')

In [102]:
ca_df.loc[:,'party'] = get_party(ca_df.name, html)

61                Doug LaMalfa
62               Audrey Denney
63               Jared Huffman
64                Dale Mensing
65              John Garamendi
66             Charlie Schaupp
67              Tom McClintock
68               Jessica Morse
69               Mike Thompson
70               Anthony Mills
71                Doris Matsui
72             Jrmar Jefferson
73                    Ami Bera
74                Andrew Grant
75                   Paul Cook
76                Tim Donnelly
77              Jerry McNerney
78             Marla Livengood
79                 Jeff Denham
80                 Josh Harder
81             Mark DeSaulnier
82             John Fitzgerald
83                Nancy Pelosi
84                 Lisa Remmer
85                 Barbara Lee
86                 Laura Wells
87               Jackie Speier
88             Cristina Osmeña
89               Eric Swalwell
90                 Rudy Peters
                ...           
136               Gil Cisneros
137     

In [103]:
ca_df

,district,name,state,chamber,party
61,District 1,Doug LaMalfa,California,house,idk
62,District 1,Audrey Denney,California,house,idk
63,District 2,Jared Huffman,California,house,idk
64,District 2,Dale Mensing,California,house,idk
65,District 3,John Garamendi,California,house,idk
66,District 3,Charlie Schaupp,California,house,idk
67,District 4,Tom McClintock,California,house,idk
68,District 4,Jessica Morse,California,house,idk
69,District 5,Mike Thompson,California,house,idk
70,District 5,Anthony Mills,California,house,idk
